In [ ]:
import mesa
from mesa import Model, Agent
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
class Participants(Agent):
    def __init__(self, model):
        super().__init__(model)
        self.has_token = False
        self.purchase_price = None # 初期状態では購入価格なし

        # 資産と効用
        self.wealth = int(self.random.normalvariate(100, 20))
        self.wealth = max(0, self.wealth)
        self.utility = int(self.random.normalvariate(80, 15))
        self.utility = max(0, self.utility)
        
        self.is_listing = False
        self.listing_price = None

        #すでにTokenを保有したことがあるか
        self.has_ever_held_token = False

    def decide_sell_price(self):
        """効用 + 20% ± 5% の価格を決定する"""
        base_price = self.utility
        price_with_markup = base_price * (1 + self.model.markup_rate)
        random_factor = self.random.uniform(0.95, 1.05)
        
        final_price = price_with_markup * random_factor
        return final_price

    def decide_and_list_if_applicable(self):
        """売り手としての行動フェーズ"""
        if self.has_token and not self.is_listing:
            if self.random.random() < self.model.listing_probability:
                price_to_list = self.decide_sell_price()
                self.model.place_sell_order(price_to_list, self)
                self.is_listing = True
                self.listing_price = price_to_list
                print(f"Agent {self.unique_id} (Utility: {self.utility}) listed token for {price_to_list:.2f}")

    def buy_if_interested(self):
        """買い手としての行動フェーズ"""
        # トークンを既に持っているエージェントは購入しない
        if self.has_ever_held_token:
            return

        # 市場に売り注文があるか確認
        if not self.model.sell_orders:
            return

        # 最も安い注文を取得
        cheapest_order = self.model.sell_orders[0]
        price = cheapest_order['price']
        seller = cheapest_order['agent']

        # 購入条件をチェック
        if self.utility >= price and self.wealth >= price:
            # 取引を実行
            self.model.execute_trade(self, seller, price)
            print(f"Agent {self.unique_id} bought token from Agent {seller.unique_id} for {price:.2f}")


class DataMarket(Model):
    def __init__(self, num, initial_token_holders, listing_probability, markup_rate=0.2, seed=None):
        super().__init__(seed=seed)
        self.num_agents = num
        self.listing_probability = listing_probability
        self.markup_rate = markup_rate
        self.steps = 0 # ステップカウンターを追加

        # AgentSet を使ってエージェントを生成・管理
        self.participants = Participants.create_agents(model=self, n=self.num_agents)

        # 初期保有者を設定
        initial_holders = self.random.sample(self.participants, k=initial_token_holders)
        for agent in initial_holders:
            agent.has_token = True
            agent.purchase_price = 100 # 前提：初期保有者は100で購入済み
            agent.has_ever_held_token = True
            #print(f"Participant {agent.unique_id} is set as an initial token holder.")

        self.sell_orders = []

    def place_sell_order(self, price, agent):
        """市場に売り注文を追加し、価格の安い順にソートする"""
        self.sell_orders.append({'price': price, 'agent': agent})
        self.sell_orders.sort(key=lambda x: x['price'])

    def execute_trade(self, buyer, seller, price):
        """売買取引を成立させる"""
        # 買い手の状態変化
        buyer.wealth -= price
        buyer.has_token = True
        buyer.purchase_price = price # 新しい購入価格を記録
        buyer.has_ever_held_token = True

        # 売り手の状態変化
        seller.wealth += price
        seller.has_token = False
        seller.is_listing = False
        seller.listing_price = None

        # 成立した注文を市場から削除
        order_to_remove = next(order for order in self.sell_orders if order['agent'] == seller)
        self.sell_orders.remove(order_to_remove)

    def step(self):
        """1ステップの処理を定義"""
        print(f"--- Step {self.steps} ---")
        
        # フェーズ1: 売り手がランダムな順序で出品を試みる
        print("Listing phase...")
        self.participants.shuffle_do("decide_and_list_if_applicable")
        
        # フェーズ2: 買い手がランダムな順序で購入を試みる
        print("Buying phase...")
        self.participants.shuffle_do("buy_if_interested")

In [62]:
# Run the simulation
model = DataMarket(num = 100, initial_token_holders= 50, listing_probability=0.1, markup_rate=0.2, seed=42)
lowest_price = {}

for i in range(100):
    model.step()
    lowest_price[model.steps] = model.sell_orders[0]["price"]


--- Step 1 ---
Listing phase...
Agent 37 (Utility: 90) listed token for 105.28
Agent 9 (Utility: 70) listed token for 87.86
Agent 23 (Utility: 94) listed token for 115.76
Agent 39 (Utility: 77) listed token for 90.00
Agent 69 (Utility: 106) listed token for 124.50
Agent 20 (Utility: 62) listed token for 72.42
Agent 12 (Utility: 48) listed token for 59.29
Buying phase...
Agent 38 bought token from Agent 12 for 59.29
Agent 7 bought token from Agent 20 for 72.42
Agent 31 bought token from Agent 9 for 87.86
Agent 5 bought token from Agent 39 for 90.00
Agent 94 bought token from Agent 37 for 105.28
--- Step 2 ---
Listing phase...
Agent 2 (Utility: 77) listed token for 95.83
Agent 89 (Utility: 78) listed token for 96.45
Agent 44 (Utility: 86) listed token for 104.30
Buying phase...
Agent 15 bought token from Agent 2 for 95.83
Agent 8 bought token from Agent 89 for 96.45
--- Step 3 ---
Listing phase...
Agent 91 (Utility: 83) listed token for 101.61
Agent 38 (Utility: 77) listed token for 91.7

In [68]:
print(lowest_price)
print(len(model.sell_orders))
print(len(model.participants.select(filter_func=lambda a: a.has_ever_held_token)))

{1: 115.76367453712112, 2: 104.29811500854922, 3: 101.60972300871882, 4: 101.60972300871882, 5: 86.65848673337406, 6: 86.65848673337406, 7: 86.65848673337406, 8: 86.65848673337406, 9: 86.65848673337406, 10: 86.65848673337406, 11: 85.6549393028059, 12: 85.6549393028059, 13: 85.46832356791417, 14: 85.46832356791417, 15: 85.46832356791417, 16: 85.46832356791417, 17: 85.46832356791417, 18: 85.46832356791417, 19: 85.46832356791417, 20: 85.46832356791417, 21: 85.46832356791417, 22: 85.46832356791417, 23: 85.46832356791417, 24: 85.46832356791417, 25: 85.46832356791417, 26: 85.46832356791417, 27: 85.46832356791417, 28: 85.46832356791417, 29: 85.46832356791417, 30: 85.46832356791417, 31: 85.46832356791417, 32: 85.46832356791417, 33: 85.46832356791417, 34: 85.46832356791417, 35: 85.46832356791417, 36: 85.46832356791417, 37: 85.46832356791417, 38: 85.46832356791417, 39: 85.46832356791417, 40: 85.46832356791417, 41: 85.46832356791417, 42: 85.46832356791417, 43: 85.46832356791417, 44: 85.4683235679